### 3.4.1 TF-IDF のサンプル

In [ ]:
# リスト 3.4.1 Wikipediaの日本百名湯記事をTF-IDFで分析

# 日本百名湯のうち、wikipediaに記事のある温泉のリスト

spa_list = ['菅野温泉','養老牛温泉','定山渓温泉','登別温泉','洞爺湖温泉','ニセコ温泉郷','朝日温泉 (北海道)',
          '酸ヶ湯温泉','蔦温泉', '花巻南温泉峡','夏油温泉','須川高原温泉','鳴子温泉郷','遠刈田温泉','峩々温泉',
           '乳頭温泉郷','後生掛温泉','玉川温泉 (秋田県)','秋ノ宮温泉郷','銀山温泉','瀬見温泉','赤倉温泉 (山形県)',
           '東山温泉','飯坂温泉','二岐温泉','那須温泉郷','塩原温泉郷','鬼怒川温泉','奥鬼怒温泉郷',
           '草津温泉','伊香保温泉','四万温泉','法師温泉','箱根温泉','湯河原温泉',
           '越後湯沢温泉','松之山温泉','大牧温泉','山中温泉','山代温泉','粟津温泉',
           '奈良田温泉','西山温泉 (山梨県)','野沢温泉','湯田中温泉','別所温泉','中房温泉','白骨温泉','小谷温泉',
           '下呂温泉','福地温泉','熱海温泉','伊東温泉','修善寺温泉','湯谷温泉 (愛知県)','榊原温泉','木津温泉',
           '有馬温泉','城崎温泉','湯村温泉 (兵庫県)','十津川温泉','南紀白浜温泉','南紀勝浦温泉','湯の峰温泉','龍神温泉',
           '奥津温泉','湯原温泉','三朝温泉','岩井温泉','関金温泉','玉造温泉','有福温泉','温泉津温泉',
           '湯田温泉','長門湯本温泉','祖谷温泉','道後温泉','二日市温泉 (筑紫野市)','嬉野温泉','武雄温泉',
           '雲仙温泉','小浜温泉','黒川温泉','地獄温泉','垂玉温泉','杖立温泉','日奈久温泉',
           '鉄輪温泉','明礬温泉','由布院温泉','川底温泉','長湯温泉','京町温泉',
           '指宿温泉','霧島温泉郷','新川渓谷温泉郷','栗野岳温泉']

In [ ]:
# wikipediaの記事の読み取り
# 2.1節参照
import wikipedia
wikipedia.set_lang("ja")

content_list = []
for spa in spa_list:
    print(spa)
    content = wikipedia.page(spa,auto_suggest=False).content
    content_list.append(content)

In [ ]:
# 形態素解析
# 2.2節参照

from janome.tokenizer import Tokenizer

# Tokenneizerインスタンスの生成
t = Tokenizer()

# 形態素解析関数の定義
def tokenize(text):
    return [token.base_form for token in t.tokenize(text) 
    if token.part_of_speech.split(',')[0] in['名詞','形容詞']]

In [ ]:
# wikipedia記事を名詞と形容詞のみとし、ブランクで分かち書き
# 2.2節参照

words_list = []
for content in content_list:
    words = ' '.join(tokenize(content))
    words = words.replace('==', '')
    words_list.append(words)

In [ ]:
# リスト 3.4.2
# TF-IDF分析の実施

# ライブラリのインポート
from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizerの初期化
vectorizer = TfidfVectorizer(min_df=1, max_df=50)

# フィーチャーベクトルの生成
features = vectorizer.fit_transform(words_list)

# 特徴語の抽出
terms = vectorizer.get_feature_names()

# フィーチャーベクトルをTF-IDF行列に変換 (numpy の ndarray 形式)
tfidfs = features.toarray()

In [ ]:
# リスト3.4.3 温泉毎の特徴語の表示

# TF-IDFの結果からi番目のドキュメントの特徴的な上位n語を取り出す関数
def extract_feature_words(terms, tfidfs, i, n):
    
    # i番目の項目のtfidfsの値リストを作成
    tfidf_array = tfidfs[i]
    
    # tfidf_arrayの値が小さい順にソートした時のインデックスリストを作る
    sorted_idx = tfidf_array.argsort()
    
    # インデックスリストを逆順に (値が大きい順のインデックスとなる)
    sorted_idx_rev = sorted_idx[::-1]
    
    # トップnのみを抽出
    top_n_idx = sorted_idx_rev[:n]  
        
    # インデックスに該当する単語リスト作成
    words = [terms[idx] for idx in top_n_idx]
    
    return words

# 結果の出力
for i in range(10):
    print(  '【' + spa_list[i] + '】' )
    for x in  extract_feature_words(terms, tfidfs, i, 10):
        print(x, end=' ')
    print()